### We have student data with marks. The goal is to calculate total marks, average percentage, topper.

Setup PySpark

In [1]:
import findspark
findspark.init()


In [2]:
import findspark
findspark.init()

import os
print(os.environ.get("JAVA_HOME"))


C:\Program Files\Java\jdk-11


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,when, round
from functools import reduce
import operator


spark = SparkSession.builder \
    .appName("Student Performance Project PySpark") \
    .getOrCreate()


Load Data from CSV file

In [4]:
print("All Students : ")
df = spark.read.csv("students.csv", header=True, inferSchema=True)
df.toPandas()


All Students : 


,Roll No,Candidate Name,EM-II,PPS,CHEM/PHY,EG/MECH,BXE/BEE
0,F101,RAI ADITYA RAJKUMAR,31,34,26,42,22
1,F102,DHOKCHAULE SHREYA RAVINDRA,35,39,25,54,21
2,F103,GORE MANOJ TUKARAM,46,45,26,58,19
3,F104,GAVALI ROHIT PUNJARAM,19,18,18,39,7
4,F105,GAIKWAD AMIT ANKUSH,22,24,29,52,18
...,...,...,...,...,...,...,...
489,F858,DHOMSE TUSHAR VILAS,12,2,12,20,4
490,F859,DAUND PIYUSH ASHOK,AB,AB,AB,AB,AB
491,F860,KARAD SHRUTI BABASAHEB,14,14,30,30,5
492,F861,BARGAJE ADITYA GOVIND,10,10,15,20,3


Check Columns name

In [5]:
print(df.columns)

['Roll No', 'Candidate Name', 'EM-II', 'PPS', 'CHEM/PHY', 'EG/MECH', 'BXE/BEE']


Absent Student : 

In [6]:
print("Absent Student in one or more Subject : ")
ab = df.filter(
    (col("EM-II") == "AB") |
    (col("PPS") == "AB") |
    (col("CHEM/PHY") == "AB") |
    (col("EG/MECH") == "AB") |
    (col("BXE/BEE") == "AB")
)
print("Count of Student :",ab.count())
ab.show(df.count(), truncate=False)

Absent Student in one or more Subject : 
Count of Student : 70
+-------+-----------------------------+-----+---+--------+-------+-------+
|Roll No|Candidate Name               |EM-II|PPS|CHEM/PHY|EG/MECH|BXE/BEE|
+-------+-----------------------------+-----+---+--------+-------+-------+
|F107   |VINCHURKAR PARTH SANTOSH     |AB   |AB |29      |19     |5      |
|F114   |CHAVAN VAIBHAV AJIT          |AB   |AB |0       |AB     |AB     |
|F121   |BATTISE MEET ASHISH          |AB   |AB |0       |AB     |AB     |
|F129   |NANDURGE KALYANI BHAGWAT     |25   |24 |40      |12     |AB     |
|F132   |PATIL BHAGYASHREE PRAFULL    |18   |19 |37      |AB     |0      |
|F133   |KOLHE RUTUJA RAJENDRA        |AB   |AB |0       |AB     |AB     |
|F137   |YADAV OMKAR SUDHIR           |10   |AB |0       |24     |3      |
|F138   |MORE JAYESH JAGDISH          |10   |12 |26      |0      |AB     |
|F144   |KALE KARAN SHYAMAL           |AB   |AB |0       |AB     |AB     |
|F154   |ARYAN KEVALDAS TURE         

In [7]:
print("Absent Student in all subjects : ")
ab_all = df.filter(
    (col("EM-II") == "AB") &
    (col("PPS") == "AB") &
    (col("CHEM/PHY") == "AB") &
    (col("EG/MECH") == "AB") &
    (col("BXE/BEE") == "AB")
)
print("Count of Student :",ab_all.count())
ab_all.show(df.count(), truncate=False)

Absent Student in all subjects : 
Count of Student : 5
+-------+------------------------+-----+---+--------+-------+-------+
|Roll No|Candidate Name          |EM-II|PPS|CHEM/PHY|EG/MECH|BXE/BEE|
+-------+------------------------+-----+---+--------+-------+-------+
|F661   |KARBHARI SRUSHTI MAHADEV|AB   |AB |AB      |AB     |AB     |
|F835   |THORAT VISHAL ANKUSH    |AB   |AB |AB      |AB     |AB     |
|F843   |OVHAL PRASHIK PREMKANT  |AB   |AB |AB      |AB     |AB     |
|F853   |SHILVANT SAHIL SACHIN   |AB   |AB |AB      |AB     |AB     |
|F859   |DAUND PIYUSH ASHOK      |AB   |AB |AB      |AB     |AB     |
+-------+------------------------+-----+---+--------+-------+-------+



Fail Student : 

In [8]:
print("Fail Student in one or more subject : ")
fail = df.filter(
    (col("EM-II") < 28 ) |
    (col("PPS") < 28 ) |
    (col("CHEM/PHY") < 28 ) |
    (col("EG/MECH") < 28 ) |
    (col("BXE/BEE") < 28 )
)
print("Count of Student :",fail.count())
fail.show(df.count(), truncate=False)

Fail Student in one or more subject : 
Count of Student : 408
+-------+------------------------------+-----+---+--------+-------+-------+
|Roll No|Candidate Name                |EM-II|PPS|CHEM/PHY|EG/MECH|BXE/BEE|
+-------+------------------------------+-----+---+--------+-------+-------+
|F101   |RAI ADITYA RAJKUMAR           |31   |34 |26      |42     |22     |
|F102   |DHOKCHAULE SHREYA RAVINDRA    |35   |39 |25      |54     |21     |
|F103   |GORE MANOJ TUKARAM            |46   |45 |26      |58     |19     |
|F104   |GAVALI ROHIT PUNJARAM         |19   |18 |18      |39     |7      |
|F105   |GAIKWAD AMIT ANKUSH           |22   |24 |29      |52     |18     |
|F106   |JADHAV SAYALI BAPU            |39   |37 |43      |44     |27     |
|F107   |VINCHURKAR PARTH SANTOSH      |AB   |AB |29      |19     |5      |
|F108   |NAGPURE AKSHATA NITIN         |28   |29 |43      |46     |21     |
|F109   |PATTEWAR SAMIKSHA BALAJI      |35   |37 |39      |29     |13     |
|F110   |PANDHE SHRAVAN SA

In [9]:
print("Fail Student In All Subject : ")
fail = df.filter(
    (col("EM-II") < 28 ) &
    (col("PPS") < 28 ) &
    (col("CHEM/PHY") < 28 ) &
    (col("EG/MECH") < 28 ) &
    (col("BXE/BEE") < 28 )
)
print("Count of Students :",fail.count())
fail.show(df.count(), truncate=False)

Fail Student In All Subject : 
Count of Students : 96
+-------+----------------------------+-----+---+--------+-------+-------+
|Roll No|Candidate Name              |EM-II|PPS|CHEM/PHY|EG/MECH|BXE/BEE|
+-------+----------------------------+-----+---+--------+-------+-------+
|F110   |PANDHE SHRAVAN SACHIN       |9    |10 |12      |15     |0      |
|F111   |JADHAO SARTHAK GAJANAN      |9    |8  |7       |22     |1      |
|F116   |BHUSHANVINODKODMUR          |9    |7  |0       |0      |0      |
|F117   |LOKHANDE GANESH SAMBHAJI    |4    |5  |0       |0      |0      |
|F135   |MANG PRASHANT BABU          |10   |9  |26      |0      |5      |
|F136   |THORBOLE DNYANESHWAR SUBHASH|9    |10 |18      |4      |0      |
|F140   |SHELKE SURAJ RAMA           |4    |5  |0       |0      |0      |
|F141   |KALE SHRAVANI NAVNATH       |12   |15 |0       |14     |8      |
|F145   |BHOKNAL PRANAV PANDHARINATH |16   |17 |9       |4      |0      |
|F147   |DUKALE PRUTHVIRAJ VIJAY     |10   |9  |10      |2

Replace AB student to 0 mark 

In [10]:

subjects = ["EM-II", "PPS", "CHEM/PHY", "EG/MECH", "BXE/BEE"]

for sub in subjects:
    df = df.withColumn(sub, when(col(sub) == "AB", 0).otherwise(col(sub).cast("int")))


Total marks & Percentage of each student : 

In [11]:
print("Total Marks & Percentage :")
df = df.withColumn("Total", reduce(operator.add, [col(sub) for sub in subjects]))

max_marks_per_subject = 70
total_max = max_marks_per_subject * len(subjects)

df = df.withColumn("Percentage", round((col("Total") / total_max) * 100, 2)) 

df.select("Roll No", "Candidate Name", "Total", "Percentage").show(df.count(), truncate=False)

Total Marks & Percentage :
+-------+------------------------------+-----+----------+
|Roll No|Candidate Name                |Total|Percentage|
+-------+------------------------------+-----+----------+
|F101   |RAI ADITYA RAJKUMAR           |155  |44.29     |
|F102   |DHOKCHAULE SHREYA RAVINDRA    |174  |49.71     |
|F103   |GORE MANOJ TUKARAM            |194  |55.43     |
|F104   |GAVALI ROHIT PUNJARAM         |101  |28.86     |
|F105   |GAIKWAD AMIT ANKUSH           |145  |41.43     |
|F106   |JADHAV SAYALI BAPU            |190  |54.29     |
|F107   |VINCHURKAR PARTH SANTOSH      |53   |15.14     |
|F108   |NAGPURE AKSHATA NITIN         |167  |47.71     |
|F109   |PATTEWAR SAMIKSHA BALAJI      |153  |43.71     |
|F110   |PANDHE SHRAVAN SACHIN         |46   |13.14     |
|F111   |JADHAO SARTHAK GAJANAN        |47   |13.43     |
|F112   |PATHAN KASHAF NAAZ SAMEER     |134  |38.29     |
|F113   |POTARE KEDAR SHIVAJI          |112  |32.0      |
|F114   |CHAVAN VAIBHAV AJIT           |0    

In [12]:
df.show(df.count(), truncate=False)

+-------+------------------------------+-----+----+--------+-------+-------+-----+----------+
|Roll No|Candidate Name                |EM-II|PPS |CHEM/PHY|EG/MECH|BXE/BEE|Total|Percentage|
+-------+------------------------------+-----+----+--------+-------+-------+-----+----------+
|F101   |RAI ADITYA RAJKUMAR           |31   |34  |26      |42     |22     |155  |44.29     |
|F102   |DHOKCHAULE SHREYA RAVINDRA    |35   |39  |25      |54     |21     |174  |49.71     |
|F103   |GORE MANOJ TUKARAM            |46   |45  |26      |58     |19     |194  |55.43     |
|F104   |GAVALI ROHIT PUNJARAM         |19   |18  |18      |39     |7      |101  |28.86     |
|F105   |GAIKWAD AMIT ANKUSH           |22   |24  |29      |52     |18     |145  |41.43     |
|F106   |JADHAV SAYALI BAPU            |39   |37  |43      |44     |27     |190  |54.29     |
|F107   |VINCHURKAR PARTH SANTOSH      |0    |0   |29      |19     |5      |53   |15.14     |
|F108   |NAGPURE AKSHATA NITIN         |28   |29  |43      |

In [13]:
topper = df.orderBy(col("Percentage").desc()).limit(1)

print("Topper :")
topper.show()


Topper :
+-------+--------------------+-----+---+--------+-------+-------+-----+----------+
|Roll No|      Candidate Name|EM-II|PPS|CHEM/PHY|EG/MECH|BXE/BEE|Total|Percentage|
+-------+--------------------+-----+---+--------+-------+-------+-----+----------+
|   F610|BHALEKAR BHAKTI B...|   63| 46|      67|     68|     64|  308|      88.0|
+-------+--------------------+-----+---+--------+-------+-------+-----+----------+

